In [149]:
import math
from scipy.stats import rankdata
import pandas as pd

In [227]:
Totals = [4, 15, 33, 61, 95] # Threshold for new row. For instance, more than 14 seats means 3 rows

sumdelegates = 14
rows = 2

# Maximum radius of spot is 0.5/rows; leave a bit of space.
radius = 0.4/rows

# Create list of centre spots
poslist = []
    
for i in range(1, rows):
    """ The relevant math is borrowed from David Richfield (slashme on GitHub)"""
        
    # Each row can contain pi/(2asin(2/(3n+4i-2))) spots, where n is the number of rows and i is the number
    # of the current row. Fill each row proportionally to the "fullness" of the diagram, up to the second-last row.
        
    J = int(float(sumdelegates) / Totals[rows-1] * math.pi/(2*math.asin(2.0/(3.0*rows+4.0*i-2.0))))
        
    # The radius of the ith row in an N-row diagram (Ri) is (3*N+4*i-2)/(4*N)
        
    R = (3.0*rows+4.0*i-2.0)/(4.0*rows)
        
    if J == 1:
        poslist.append([math.pi/2.0, 1.75*R, R])
    else:
        for j in range(J):
            # The angle to a spot is n.(pi-2sin(r/Ri))/(Ni-1)+sin(r/Ri) where Ni is the number in the arc
            # x=R.cos(theta) + 1.75
            # y=R.sin(theta)
                
            angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J)-1.0)+math.sin(radius/R)   
            poslist.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])

                
# Now whatever seats are left go into the outside row:
J_2 = sumdelegates-len(poslist)
R = (7.0*rows-2.0)/(4.0*rows)
if J_2 == 1:
    poslist.append([math.pi/2.0, 1.75*R, R])
else:
    for j in range(J_2):
        angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J_2)-1.0)+math.sin(radius/R)
        poslist.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])
poslist.sort(reverse = True)

# Pure seats allocation -> add the party that holds the seat
dataframe = pd.DataFrame(data=poslist, columns = ['Angle', 'x', 'y'])
party = pd.Series(['A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D'])
df = pd.concat([dataframe, party], axis = 1, ignore_index=True)
df.columns = ['Rank', 'x', 'y', 'party']
df

,Rank,x,y,party
0,3.008654,0.263235,0.198821,A
1,2.942923,0.769670,0.197365,A
2,2.649190,0.428201,0.709117,A
3,2.289725,0.762132,1.128767,B
4,2.256860,1.116504,0.773746,B
5,1.930261,1.222341,1.404128,B
6,1.570796,1.750000,1.500000,B
7,1.570796,1.750000,1.000000,B
8,1.211332,2.277659,1.404128,C
9,0.884733,2.383496,0.773746,C


In [206]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

plotly.tools.set_credentials_file(username='marius92', api_key='4naxu3XiGZTGniWO02z1')

In [255]:
data = []
color = ['#0D76BF', '#43B02A', '#F93822', '#3E332E']
list_parties= df.party.unique()

for i in range(len(list_parties)):
    
    party_now = list_parties[i]
    color_now = color[i]
    
    trace = go.Scatter(
        x= df[df['party'].str.match(party_now)]['x'], 
        y= df[df['party'].str.match(party_now)]['y'],
        mode='markers',
        name = party_now,
        marker = dict(
                size = 24,
                color = color_now),
        hoverinfo = 'text',
        hovertext = party_now, # Change to forecasted politician/or just number of seat for party ?
    )
    data.append(trace)

summe = len(df['party'])    
_sum = str(summe)

layout = go.Layout(
    legend = dict(orientation="h"),
    xaxis = dict(showgrid=False, zeroline=False, visible = False),
    yaxis = dict(showgrid=False, zeroline=False, visible = False),
    font = dict(),
    annotations = [
        go.layout.Annotation(
            x = df['x'][7], # middle element of x column
            y = df['y'][0], # first element of y column
            text = _sum + " Sitze", # Summe aller Sitze
            font = dict(
                size = 24            
            ),
            showarrow = False,
        )
    ]
)

fig = dict(data=data, layout = layout)

py.iplot(fig, filename = 'test')